# 09. Spark Streaming

9.1 Extend the Spark Dataframe Example for counting HTTP response code in the NASA log files to Spark Structured Streaming. Use the [DataStreamReader] (https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.streaming.DataStreamReader) API and the output mode "update"!

* https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html
* https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.streaming
* https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.streaming.DataStreamReader    
* `spark_streaming.py`
    
    
9.1 Add an additional copy of the NASA file to your Streaming data source directory! What do you observer?

9.2 Create a subset of the data (10 rows) and slightly modify it! Add it to the source directory!

In [3]:
import os
import sys
import time
import datetime
import logging
import numpy as np
import socket
import re
from subprocess import check_output

from pyspark.mllib.linalg import Vectors
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.clustering import StreamingKMeans
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
from pyspark.sql.types import StructType
from pyspark.ml.feature import FeatureHasher
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml import Pipeline
from pyspark.sql import functions as F
from pyspark.sql.streaming import DataStreamReader

#######################################################################################
# CONFIGURATIONS
# Get current cluster setup from work directory
STREAMING_WINDOW=60

# Initialize PySpark
SPARK_MASTER="local[1]"
#SPARK_MASTER="spark://mpp3r03c04s06.cos.lrz.de:7077"
APP_NAME = "PySpark Lecture"
os.environ["PYSPARK_PYTHON"] = "/opt/tljh/user/bin/python"

# If there is no SparkSession, create the environment
try:
    sc and spark
except NameError as e:
    import pyspark
    import pyspark.sql
    conf=pyspark.SparkConf().set("spark.cores.max", "4")
    sc = pyspark.SparkContext(master=SPARK_MASTER, conf=conf)
    spark = pyspark.sql.SparkSession(sc).builder.appName(APP_NAME).getOrCreate()

In [6]:
stream_reader = spark.readStream

In [22]:
data = stream_reader.load("/opt/data/nasa/", format="text")

In [13]:
data.map(lambda x: x.split(' ')).map(lambda x: int(x[-2]) if len(x) >= 2 else 0).countByValue()

AttributeError: 'DataFrame' object has no attribute 'map'

In [23]:
data.writeStream \
    .format("console") \
    .start().awaitTermination()

StreamingQueryException: 'Writing job aborted.\n=== Streaming Query ===\nIdentifier: [id = 774e0dab-08bb-4e52-bafd-c728da234bd8, runId = 82f7d3b8-3650-4d1c-9d27-7bd0ec0da01f]\nCurrent Committed Offsets: {}\nCurrent Available Offsets: {FileStreamSource[file:/opt/data/nasa]: {"logOffset":0}}\n\nCurrent State: ACTIVE\nThread State: RUNNABLE\n\nLogical Plan:\nFileStreamSource[file:/opt/data/nasa]'